In [2]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/Users/Kanwar/Desktop/assignment/dataset.csv')

# Filter BUY and SELL data
buy_data = data[data['type'] == 'BUY']
sell_data = data[data['type'] == 'SELL']

# Merge BUY and SELL data on Container_Number
merged_data = pd.merge(sell_data, buy_data, on='Container_Number', suffixes=('_sell', '_buy'))

# Calculate margin
merged_data['margin'] = merged_data['priceQuantity_sell'] - merged_data['priceQuantity_buy'] - merged_data['logisticCostPerTon_buy'].fillna(0)

# Calculate total margin, total purchase amount, total sale amount
total_margin = merged_data['margin'].sum()
total_purchase_amount = (merged_data['priceQuantity_buy'] * merged_data['quantity_buy']).sum()
total_sale_amount = (merged_data['priceQuantity_sell'] * merged_data['quantity_sell']).sum()

# Calculate margin per product
margin_per_product = merged_data.groupby('product name_sell')['margin'].sum().reset_index()

# Calculate margin per customer
margin_per_customer = merged_data.groupby('name_sell')['margin'].sum().reset_index()

# Calculate margin per supplier
margin_per_supplier = merged_data.groupby('name_buy')['margin'].sum().reset_index()

# Calculate margin per ton
merged_data['margin_per_ton'] = merged_data['margin'] / merged_data['quantity_sell']

# Identify worst trades
worst_trades = merged_data.nsmallest(10, 'margin')

# Identify worst customers
worst_customers = margin_per_customer.nsmallest(3, 'margin')

# Identify worst suppliers
worst_suppliers = margin_per_supplier.nsmallest(3, 'margin')

# Display the results
print("Total Margin:", total_margin)
print("Total Purchase Amount:", total_purchase_amount)
print("Total Sale Amount:", total_sale_amount)
print("Margin per Product:\n", margin_per_product)
print("Margin per Customer:\n", margin_per_customer)
print("Margin per Supplier:\n", margin_per_supplier)
print("Worst Trades:\n", worst_trades)
print("Worst Customers:\n", worst_customers)
print("Worst Suppliers:\n", worst_suppliers)

Total Margin: 6922.94020793
Total Purchase Amount: 730340.6660901913
Total Sale Amount: 810230.7586557053
Margin per Product:
                                     product name_sell       margin
0                      Bright colored paper documents   293.080851
1                                         Brown glass     2.000000
2   Cardboard packaging for cigarettes (Czech Repu...    40.000000
3        Cardboard packaging for cigarettes (Germany)    25.800000
4                                         Clear glass   -15.000000
5              Coated paper for high-quality printing    20.000000
6     Coated paper for magazine or newspaper printing     7.000000
7                                    Continuous forms    55.000000
8                                               Cores    72.000000
9                                        Deinked pulp   166.000000
10                                        Green glass     4.000000
11                                         Grey board   141.800002
12

In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)

# Create figures for the dashboard
fig_total_margin = px.bar(x=['Total Margin'], y=[total_margin], labels={'x': 'Metric', 'y': 'Amount'}, title='Total Margin')
fig_total_purchase_amount = px.bar(x=['Total Purchase Amount'], y=[total_purchase_amount], labels={'x': 'Metric', 'y': 'Amount'}, title='Total Purchase Amount')
fig_total_sale_amount = px.bar(x=['Total Sale Amount'], y=[total_sale_amount], labels={'x': 'Metric', 'y': 'Amount'}, title='Total Sale Amount')
fig_margin_per_product = px.bar(margin_per_product, x='product name_sell', y='margin', labels={'product name_sell': 'Product', 'margin': 'Margin'}, title='Margin per Product')
fig_margin_per_customer = px.bar(margin_per_customer, x='name_sell', y='margin', labels={'name_sell': 'Customer', 'margin': 'Margin'}, title='Margin per Customer')
fig_margin_per_supplier = px.bar(margin_per_supplier, x='name_buy', y='margin', labels={'name_buy': 'Supplier', 'margin': 'Margin'}, title='Margin per Supplier')
fig_worst_trades = px.bar(worst_trades, x='Container_Number', y='margin', labels={'Container_Number': 'Container Number', 'margin': 'Margin'}, title='Worst Trades')
fig_worst_customers = px.bar(worst_customers, x='name_sell', y='margin', labels={'name_sell': 'Customer', 'margin': 'Margin'}, title='Worst Customers')
fig_worst_suppliers = px.bar(worst_suppliers, x='name_buy', y='margin', labels={'name_buy': 'Supplier', 'margin': 'Margin'}, title='Worst Suppliers')

# Define the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children='Margin and Anomaly Report Dashboard'),

    dcc.Graph(
        id='total-margin',
        figure=fig_total_margin
    ),
    dcc.Graph(
        id='total-purchase-amount',
        figure=fig_total_purchase_amount
    ),
    dcc.Graph(
        id='total-sale-amount',
        figure=fig_total_sale_amount
    ),
    dcc.Graph(
        id='margin-per-product',
        figure=fig_margin_per_product
    ),
    dcc.Graph(
        id='margin-per-customer',
        figure=fig_margin_per_customer
    ),
    dcc.Graph(
        id='margin-per-supplier',
        figure=fig_margin_per_supplier
    ),
    dcc.Graph(
        id='worst-trades',
        figure=fig_worst_trades
    ),
    dcc.Graph(
        id='worst-customers',
        figure=fig_worst_customers
    ),
    dcc.Graph(
        id='worst-suppliers',
        figure=fig_worst_suppliers
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)